In [ ]:

import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt

plt.style.use("seaborn")


In [ ]:

# Image loading and simple visualization
def load_image(path):
    image = cv.imread(path, 0)
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.show()
    return image

image_path = 'line1.jpg'
image = load_image(image_path)


In [ ]:

# Basic function to compute min and max of an image
def image_min_max(image):
    return np.min(image), np.max(image)

min_val, max_val = image_min_max(image)
print("Min and Max Values:", min_val, max_val)


In [ ]:

# Plot a simple histogram of image pixel values
def plot_histogram(data):
    plt.hist(data.flatten(), bins=50, color='blue')
    plt.title('Histogram of Image Pixel Values')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.show()

plot_histogram(image)
